In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install datasets unsloth

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.4/46.4 kB 3.3 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 218.3/218.3 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 32.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.0/129.0 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.5/31.5 MB 68.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8

In [ ]:
from datasets import load_dataset
from unsloth import FastLanguageModel
import torch

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [ ]:
max_seq_length = 2048
dtype = None
load_in_4bit = True

In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Phi-3.5-mini-instruct",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

==((====))==  Unsloth 2025.4.5: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.3.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.26G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/140 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/3.37k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.4.5 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [ ]:
import json
from datasets import Dataset

dataset_path = "/content/drive/MyDrive/proccessed_data_5k.jsonl"

conversations = []
with open(dataset_path, "r") as f:
    for line in f:
        example = json.loads(line.strip())

        chat = []
        for msg in example["messages"]:
            role = msg["role"]
            content = msg["content"]

            if role == "system":
                continue

            if role == "user":
                chat.append({"from": "human", "value": content})
            elif role == "assistant":
                chat.append({"from": "gpt", "value": content})

        if chat:
            conversations.append({"conversations": chat})

dataset = Dataset.from_list(conversations)

train_test_split = dataset.train_test_split(test_size=0.2)

train_dataset = train_test_split["train"]
test_dataset = train_test_split["test"]

# dataset_dict = DatasetDict({
#     "train": train_dataset,
#     "test": test_dataset
# })

print(dataset[0])


{'conversations': [{'from': 'human', 'value': 'Hãy viết một bài blog ngắn về lợi ích của việc đọc sách.'}, {'from': 'gpt', 'value': 'Bài viết:  Việc đọc sách có rất nhiều lợi ích cho con người. Đây là một hoạt động giáo dục và giải trí hữu ích, giúp chúng ta mở rộng kiến thức và hiểu biết về thế giới xung quanh.  Một trong những lợi ích đáng kể của việc đọc sách là cải thiện khả năng ngôn ngữ của chúng ta. Khi đọc sách, chúng ta tiếp xúc với các từ ngữ mới, cấu trúc câu phức tạp và ngữ cảnh sử dụng. Điều này giúp chúng ta mở rộng vốn từ vựng và cải thiện khả năng diễn đạt bằng ngôn ngữ.  Ngoài ra, đọc sách cũng có tác động tích cực đến trí tuệ và tư duy của con người. Việc đọc sách đòi hỏi chúng ta tập trung, tư duy logic và sáng tạo. Chúng ta phải tưởng tượng và hình dung các tình huống, nhân vật và cốt truyện. Điều này giúp phát triển não bộ và khả năng tư duy sáng tạo của chúng ta.  Không chỉ có lợi ích về trí tuệ, việc đọc sách còn giúp chúng ta giải tỏa stress và thư giãn. Khi đọc

In [ ]:
from unsloth.chat_templates import get_chat_template

# Set your tokenizer chat template
tokenizer = get_chat_template(
    tokenizer,
    chat_template="phi-3",  # Or another supported template like "mistral", "llama", etc.
    mapping={"role": "from", "content": "value", "user": "human", "assistant": "gpt"}
)

# Formatting function for SFTTrainer
def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [
        tokenizer.apply_chat_template(
            convo, tokenize=False, add_generation_prompt=False
        )
        for convo in convos
    ]
    return {"text": texts}

train_dataset = dataset.map(formatting_prompts_func, batched = True,)
test_dataset = dataset.map(formatting_prompts_func, batched = True,)

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

In [ ]:
train_dataset[3]["conversations"]

[{'from': 'human',
  'value': 'Chỉ ra một khả năng tích cực của người ít được biết đến, dựa trên một đặc điểm cụ thể.'},
 {'from': 'gpt',
  'value': 'Dù ít nói, người đó thường mang lại những ý kiến sắc sảo và chín chắn khi tham gia cuộc trò chuyện.'}]

In [ ]:
train_dataset[3]["text"]

'<|user|>\nChỉ ra một khả năng tích cực của người ít được biết đến, dựa trên một đặc điểm cụ thể.<|end|>\n<|assistant|>\nDù ít nói, người đó thường mang lại những ý kiến sắc sảo và chín chắn khi tham gia cuộc trò chuyện.<|end|>\n'

In [ ]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"from": "human", "value": "xin vui lòng mô tả Việt Nam"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 128, use_cache = True)

<|user|> xin vui lòng mô tả Việt Nam<|end|><|assistant|> Hello! Một bài đọc về Việt Nam. Đây là một nền không gian rất phong phú, rất rộng rãi, rất phong phú về môi giới, rất phong phú về môi giới, rất phong phú về môi giới, rất phong phú về môi giới, rất phong phú về m


In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

# FastLanguageModel.for_trainining(model)

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/5000 [00:00<?, ? examples/s]

In [ ]:
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.741 GB.
3.648 GB of memory reserved.


In [ ]:
import os
# Disable torch.compile in Unsloth
os.environ["UNSLOTH_DISABLE_TORCH_COMPILE"] = "1"
# Disable Triton debug info that causes PassManager::run failures
os.environ["TRITON_DISABLE_LINE_INFO"] = "1"

import torch
# Avoid float32 kernel issues
torch.set_float32_matmul_precision("medium")


In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 5,000 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 29,884,416/4,000,000,000 (0.75% trained)


RuntimeError: PassManager::run failed

In [ ]:
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

182.9863 seconds used for training.
3.05 minutes used for training.
Peak reserved memory = 3.648 GB.
Peak reserved memory for training = 0.0 GB.
Peak reserved memory % of max memory = 24.747 %.
Peak reserved memory for training % of max memory = 0.0 %.


In [ ]:
 FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"from": "human", "value": "Tiếp tục chuỗi fibonacci: 1, 1, 2, 3, 5, 8"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 128, use_cache = True)

IndexError: too many indices for tensor of dimension 2